## Weekly Data Updater

In [1]:
import pandas as pd

In [2]:
WEEK = 11
YEAR = 2022

positions = ['qb', 'rb', 'wr', 'te', 'dst']

In [3]:
data = {}
for pos in positions:
    url = f'https://www.fantasypros.com/nfl/stats/{pos}.php?year={YEAR}&scoring=HALF&range=week&week={WEEK}'
    data[pos] = pd.read_html(url)[0]

In [4]:
col_mapper = {
    'qb':['WeekRank', 'Player', 'PassCMP', 'PassATT', 'PassPCT', 'PassYDS', 'PassY/A', 'PassTD', 'PassInt', 'Sacks', 
           'RushAtt', 'RushYDS', 'RushTD', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'rb':['WeekRank', 'Player', 'RushATT', 'RushYDS', 'RushY/A', 'RushLG', 'Rush20+', 'RushTD',
           'RecREC', 'RecTGT', 'RecYDS', 'RecY/R', 'RecTD', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'wr':['WeekRank', 'Player', 'RecREC', 'RecTGT', 'RecYDS', 'RecY/R', 'RecLG', 'Rec20+', 'RecTD',
           'RushATT', 'RushYDS', 'RushTD', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'te':['WeekRank', 'Player', 'RecREC', 'RecTGT', 'RecYDS', 'RecY/R', 'RecLG', 'Rec20+', 'RecTD',
           'RushATT', 'RushYDS', 'RushTD', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'dst':['WeekRank', 'Player', 'SACK', 'INT', 'FR', 'FF', 'DEF TD', 
            'SFTY', 'SPC TD', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'k':['WeekRank', 'Player', 'FG', 'FGA', 'PCT', 'LG', '1-19', '20-29', '30-39',
          '40-49', '50+', 'XPT', 'XPA', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'dl':['WeekRank', 'Player', 'TACKLE', 'ASSIST', 'SACK', 'PD', 'INT', 'FF', 'FR',
           'DEF TD', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'db':['WeekRank', 'Player', 'TACKLE', 'ASSIST', 'SACK', 'PD', 'INT', 'FF', 'FR',
           'DEF TD', 'G', 'FPTS', 'FPTS/G', 'ROST'],
    'lb':['WeekRank', 'Player', 'TACKLE', 'ASSIST', 'SACK', 'PD', 'INT', 'FF', 'FR',
           'DEF TD', 'G', 'FPTS', 'FPTS/G', 'ROST']
}

In [5]:
len(data['qb'].columns)

18

In [6]:
sheets = {}
for pos in data.keys():
    temp = data[pos].copy()
    temp.columns = col_mapper[pos]
    temp['Season'] = YEAR
    temp['WeekNum'] = WEEK
    sheets[pos] = temp[temp['G']!=0]

In [7]:
# write one week of actuals
with pd.ExcelWriter(f'Weekly Actuals/{YEAR}-{str(WEEK).zfill(2)} FP Actuals.xlsx') as writer:
    for i in sheets.keys():
        sheets[i].to_excel(writer, sheet_name=i, index=False)

In [8]:
# load existing data
existing = dict((pos, pd.read_excel('FantasyPros NFL Historical.xlsx', sheet_name=pos)) for pos in positions)
len(existing.keys())

5

In [9]:
# sort existing data
for pos in positions[:5]:
    existing[pos] = existing[pos].sort_values(['Season', 'WeekNum', 'WeekRank'])

In [10]:
# # update cumulative workbook
# cat_sheets = {}
# for pos in data.keys():
#     cat  = pd.concat([existing[pos], sheets[pos]], axis=0)
#     cat_sheets[pos] = cat[cat['G']!=0]
    
# with pd.ExcelWriter('FantasyPros NFL Historical.xlsx') as writer:
#     for i in cat_sheets.keys():
#         cat_sheets[i].to_excel(writer, sheet_name=i, index=False)